In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import gc

import warnings
warnings.filterwarnings('ignore')


In [3]:
# group by peptides
df_by_peptide = df_all.groupby(['visit_id','Peptide'])['PeptideAbundance'].mean().reset_index()


NameError: name 'df_all' is not defined

# EDA

### Summary

- there are 248 patients (but a total of 771 in the supplemental dataset)
- patients have more than 1 record for each visit
  - there are ~1113 visit ids, so each patient visits around 4-5 times
- sampling random patient, there are around 200 proteins found in each of their sample visits

#### Proteins and Peptides datasets
- categorical data: "UniProt"(protein) and "Peptide"
- continuous data: "NPX" and "PeptideAbundance"

#### Clinical and Supplemental datasets
- categorical data: "upd23b_clinical_state_on_medication"
- continuous data: "updrs_1" to "updrs_4"

In [5]:
#load dataset
proteins = pd.read_csv("amp-parkinsons-disease-progression-prediction/train_proteins.csv")
peptides = pd.read_csv("amp-parkinsons-disease-progression-prediction/train_peptides.csv")
clinical = pd.read_csv("amp-parkinsons-disease-progression-prediction/train_clinical_data.csv")
supplemental = pd.read_csv("amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv")

proteins.shape,peptides.shape,clinical.shape, supplemental.shape

((232741, 5), (981834, 6), (2615, 8), (2223, 8))

In [34]:
## KEEP?
# combine clinical and supplemental datasets
clinical_supplemental = pd.concat([clinical, supplemental]).reset_index(drop=True)
clinical_supplemental.shape

(4838, 8)

In [9]:
proteins.sample(10)

,visit_id,visit_month,patient_id,UniProt,NPX
35187,57416_0,0,57416,Q7Z3B1,77401.2
199868,57468_48,48,57468,Q8IWV7,100322.0
142859,55_36,36,55,P04207,48765.0
91146,41444_12,12,41444,P01857,6326660.0
169465,57416_36,36,57416,P16870,209883.0
228487,39144_84,84,39144,P35542,46908.1
36481,58653_0,0,58653,P05060,105985.0
125559,31121_24,24,31121,P43652,344616.0
141206,62723_24,24,62723,P07339,281284.0
193776,44001_48,48,44001,P02652,428899.0


In [24]:
peptides.sample(10)

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
104931,40874_0,0,40874,P08603,DGWSAQPTC(UniMod_4)IK,51272.40
851921,63889_48,48,63889,Q14515,DQGNQEQDPNISNGEEEEEKEPGEVGTHNDNQER,13807.10
493092,18183_24,24,18183,P02649,KVEQAVETEPEPELR,2360880.00
133868,53103_0,0,53103,P02747,FQSVFTVTR,120655.00
162915,62329_0,0,62329,P36980,LVYPSC(UniMod_4)EEK,8186.86
972642,59550_84,84,59550,P36222,LVMGIPTFGR,286193.00
408072,52266_12,12,52266,P02671,QFTSSTSYNR,6530.43
68813,25911_0,0,25911,P08603,SC(UniMod_4)DNPYIPNGDYSPLR,19068.00
577988,56675_24,24,56675,P01009,LYHSEAFTVNFGDTEEAKK,36322.70
48995,20216_0,0,20216,P10645,GLSAEPGWQAK,66677.00


In [7]:
clinical.sample(10)

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
419,12755_18,12755,18,2.0,3.0,7,NaN,NaN
787,18560_60,18560,60,1.0,3.0,0,0.0,On
193,7051_12,7051,12,5.0,8.0,2,0.0,NaN
2145,55256_9,55256,9,4.0,1.0,17,NaN,NaN
2250,56675_42,56675,42,5.0,6.0,4,0.0,On
1223,27893_24,27893,24,7.0,15.0,16,0.0,On
1665,41444_36,41444,36,3.0,8.0,9,0.0,Off
1105,26104_24,26104,24,9.0,6.0,16,0.0,On
58,3636_30,3636,30,9.0,4.0,21,0.0,On
145,5742_0,5742,0,5.0,7.0,6,NaN,NaN


In [10]:
supplemental.sample(10)

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
7,BF-1005_M0,65382,0,NaN,NaN,6,NaN,NaN
475,SU-32090_M12,27867,12,3.0,1.0,2,0.0,On
2212,SY-PDZV652BE6_M36,16656,36,4.0,7.0,9,0.0,On
1398,SU-33088_M6,4005,6,7.0,2.0,58,NaN,NaN
571,SU-32166_M30,33978,30,0.0,1.0,21,0.0,On
1025,SU-32648_M0,14077,0,9.0,10.0,6,NaN,On
1255,SU-32854_M30,29662,30,6.0,10.0,21,NaN,On
2198,SY-PDZH104KR2_M36,37821,36,1.0,11.0,9,0.0,On
1349,SU-32926_M0,62815,0,7.0,4.0,6,NaN,NaN
1818,SY-PDFB947NXY_M0,19388,0,10.0,3.0,6,0.0,NaN


In [20]:
print(f"number of unique patient ids in proteins: {proteins.patient_id.nunique()}")
print(f"number of unique patient ids in peptides: {peptides.patient_id.nunique()}")
print(f"number of unique patient ids in clinical: {clinical.patient_id.nunique()}")
print(f"number of unique patient ids in supplemental: {supplemental.patient_id.nunique()}")

number of unique patient ids in proteins: 248
number of unique patient ids in peptides: 248
number of unique patient ids in clinical: 248
number of unique patient ids in supplemental: 771


In [19]:
print(f"number of unique visits per month in proteins: {proteins.visit_month.nunique()}")
print(f"number of unique visits per month in peptides: {peptides.visit_month.nunique()}")
print(f"number of unique visits per month in clinical: {clinical.visit_month.nunique()}")
print(f"number of unique visits per month in supplemental: {supplemental.visit_month.nunique()}")

number of unique visits per month in proteins: 15
number of unique visits per month in peptides: 15
number of unique visits per month in clinical: 17
number of unique visits per month in supplemental: 8


In [22]:
# average number of visits per patient
print(f"average number of visits per patient in proteins: {proteins.visit_id.nunique()/proteins.patient_id.nunique()}")
print(f"average number of visits per patient in peptides: {peptides.visit_id.nunique()/peptides.patient_id.nunique()}")
print(f"average number of visits per patient in clinical: {clinical.visit_id.nunique()/clinical.patient_id.nunique()}")
print(f"average number of visits per patient in supplemental: {supplemental.visit_id.nunique()/supplemental.patient_id.nunique()}")


average number of visits per patient in proteins: 4.487903225806452
average number of visits per patient in peptides: 4.487903225806452
average number of visits per patient in clinical: 10.544354838709678
average number of visits per patient in supplemental: 2.8832684824902723


In [37]:
print(f"The months recorded (looks like every 3, then every 6, then 12): {proteins.visit_month.unique()}")
print(f"The number of months recorded: {proteins.visit_month.nunique()}")


The months recorded (looks like every 3, then every 6, then 12): [  0   3   6  12  18  24  30  36  48  54  60  72  84  96 108]
The number of months recorded: 15


In [28]:
# find number of missing values, max and min, data types, #unique values for each colum

def summary(df):
    print(f'data shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['#missing'] = df.isnull().sum().values * 100
    summ['%missing'] = df.isnull().sum().values / len(df)
    summ['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    
    return summ

In [30]:
summary(df_all)

data shape: (981834, 7)


,data type,#missing,%missing,#unique,min,max
visit_id,object,0,0.0,1113,NaN,NaN
visit_month,int64,0,0.0,15,0.0,108.0
patient_id,int64,0,0.0,248,55.0,65043.0
UniProt,object,0,0.0,227,NaN,NaN
NPX,float64,0,0.0,218795,84.6082,613851000.0
Peptide,object,0,0.0,968,NaN,NaN
PeptideAbundance,float64,0,0.0,738931,10.9985,178752000.0


# Data Preprocessing

In [38]:
# merge proteins and peptides dataset based on 'visit_id' and 'UniProt id'
proteins_peptides = proteins.merge(peptides[['visit_id', 'UniProt', 'Peptide','PeptideAbundance']], on = ['visit_id','UniProt'], how = 'left')
proteins_peptides

,visit_id,visit_month,patient_id,UniProt,NPX,Peptide,PeptideAbundance
0,55_0,0,55,O00391,11254.3,NEQEQPLGQWHLS,11254.30
1,55_0,0,55,O00533,732430.0,GNPEPTFSWTK,102060.00
2,55_0,0,55,O00533,732430.0,IEIPSSVQQVPTIIK,174185.00
3,55_0,0,55,O00533,732430.0,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.90
4,55_0,0,55,O00533,732430.0,SMEQNGPGLEYR,30838.70
...,...,...,...,...,...,...,...
981829,58648_108,108,58648,Q9UHG2,369437.0,ILAGSADSEGVAAPR,202820.00
981830,58648_108,108,58648,Q9UKV8,105830.0,SGNIPAGTTVDTK,105830.00
981831,58648_108,108,58648,Q9Y646,21257.6,LALLVDTVGPR,21257.60
981832,58648_108,108,58648,Q9Y6R7,17953.1,AGC(UniMod_4)VAESTAVC(UniMod_4)R,5127.26


In [43]:
# group by uniprot id
df_by_uniprot = proteins_peptides.groupby(['visit_id','UniProt'])['NPX'].mean().reset_index()
df_by_uniprot

,visit_id,UniProt,NPX
0,10053_0,O00391,9104.27
1,10053_0,O00533,402321.00
2,10053_0,O14773,7150.57
3,10053_0,O14791,2497.84
4,10053_0,O15240,83002.90
...,...,...,...
232736,942_6,Q9UHG2,306621.00
232737,942_6,Q9UKV8,82335.50
232738,942_6,Q9UNU6,24018.70
232739,942_6,Q9Y646,18939.50


In [44]:
# group by peptides
df_by_peptide = proteins_peptides.groupby(['visit_id','Peptide'])['PeptideAbundance'].mean().reset_index()
df_by_peptide

,visit_id,Peptide,PeptideAbundance
0,10053_0,AADDTWEPFASGK,6580710.0
1,10053_0,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,31204.4
2,10053_0,AAFTEC(UniMod_4)C(UniMod_4)QAADK,7735070.0
3,10053_0,AAVYHHFISDGVR,46620.3
4,10053_0,ADDKETC(UniMod_4)FAEEGK,236144.0
...,...,...,...
981829,942_6,YVNKEIQNAVNGVK,80001.2
981830,942_6,YWGVASFLQK,79661.9
981831,942_6,YYC(UniMod_4)FQGNQFLR,573300.0
981832,942_6,YYTYLIMNK,48005.8


In [42]:
# pivot groupby table for uniprot dataset
df_uniprot = df_by_uniprot.pivot(index='visit_id',columns = 'UniProt', values = 'NPX').rename_axis(columns=None).reset_index()
df_uniprot


,visit_id,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
0,10053_0,9104.27,402321.0,NaN,NaN,7150.57,2497.84,83002.9,15113.6,167327.0,...,NaN,9469.45,94237.6,NaN,23016.0,177983.0,65900.0,15382.0,NaN,19017.40
1,10053_12,10464.20,435586.0,NaN,NaN,NaN,NaN,197117.0,15099.1,164268.0,...,NaN,14408.40,NaN,NaN,28537.0,171733.0,65668.1,NaN,9295.65,25697.80
2,10053_18,13235.70,507386.0,7126.96,24525.7,NaN,2372.71,126506.0,16289.6,168107.0,...,317477.0,38667.20,111107.0,NaN,37932.6,245188.0,59986.1,10813.3,NaN,29102.70
3,10138_12,12600.20,494581.0,9165.06,27193.5,22506.10,6015.90,156313.0,54546.4,204013.0,...,557904.0,44556.90,155619.0,14647.90,36927.7,229232.0,106564.0,26077.7,21441.80,7642.42
4,10138_24,12003.20,522138.0,4498.51,17189.8,29112.40,2665.15,151169.0,52338.1,240892.0,...,NaN,47836.70,177619.0,17061.10,25510.4,176722.0,59471.4,12639.2,15091.40,6168.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,8699_24,9983.00,400290.0,24240.10,NaN,16943.50,6303.17,77493.6,46435.3,254247.0,...,NaN,25690.60,NaN,6859.82,19106.7,121161.0,113872.0,14413.9,28225.50,8062.07
1109,942_12,6757.32,360858.0,18367.60,14760.7,18603.40,1722.77,86847.4,37741.3,212132.0,...,45742.3,33518.60,94049.7,13415.70,21324.7,234094.0,82410.4,19183.7,17804.10,12277.00
1110,942_24,NaN,352722.0,22834.90,23393.1,16693.50,1487.91,114772.0,36095.7,185836.0,...,180475.0,29770.60,95949.9,11344.40,23637.6,256654.0,76931.9,19168.2,19215.90,14625.60
1111,942_48,11627.80,251820.0,22046.50,26360.5,22440.20,2117.43,82241.9,30146.6,167633.0,...,197987.0,29283.80,121696.0,19169.80,16724.9,232301.0,96905.9,21120.9,14089.80,16418.50


In [34]:
# pivot groupby table for peptide dataset
df_peptide = df_by_peptide.pivot(index='visit_id',columns = 'Peptide', values = 'PeptideAbundance').rename_axis(columns=None).reset_index()
df_peptide.sample(5)


,visit_id,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,AANEVSSADVK,AATGEC(UniMod_4)TATVGKR,AATVGSLAGQPLQER,AAVYHHFISDGVR,ADDKETC(UniMod_4)FAEEGK,ADDKETC(UniMod_4)FAEEGKK,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
310,23636_36,8340370.0,37626.0,9206360.0,21920.4,16533.90,18312.0,45248.8,507172.0,8434700.0,...,201752.0,6765.62,5572700.0,72490.6,816316.0,40052.9,153741.0,552240.0,47338.6,13092.5
1103,8344_24,7254380.0,39743.8,8185820.0,19223.4,17588.80,11032.0,36362.1,503480.0,6437940.0,...,212614.0,3801.31,3827320.0,14388.7,211949.0,50382.4,114154.0,396690.0,40111.2,24491.7
779,5178_24,6137290.0,79552.2,14751500.0,30336.3,11632.90,151495.0,54960.1,809795.0,5037490.0,...,289760.0,17919.20,3609270.0,NaN,241446.0,100705.0,157346.0,417611.0,78377.5,36280.9
431,28327_6,6887080.0,37505.6,10267600.0,28026.8,17704.50,74757.0,43162.8,54512.9,5602390.0,...,198308.0,8517.34,4423710.0,67557.5,418102.0,80492.3,162787.0,421868.0,40905.1,NaN
397,27468_36,6259060.0,36605.0,8722450.0,20383.7,8854.39,65851.9,NaN,57170.4,4782070.0,...,277114.0,10560.50,NaN,84813.2,460885.0,142576.0,112294.0,523755.0,35042.7,20323.5


In [37]:
# add patient id and visit month
df_uniprot[['patient_id','visit_month']] = df_uniprot.visit_id.str.split("_", expand=True)
df_peptide[['patient_id','visit_month']] = df_peptide.visit_id.str.split("_", expand=True)

In [38]:
df_uniprot

,visit_id,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,...,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7,patient_id,visit_month
0,10053_0,9104.27,402321.0,NaN,NaN,7150.57,2497.84,83002.9,15113.6,167327.0,...,94237.6,NaN,23016.0,177983.0,65900.0,15382.0,NaN,19017.40,10053,0
1,10053_12,10464.20,435586.0,NaN,NaN,NaN,NaN,197117.0,15099.1,164268.0,...,NaN,NaN,28537.0,171733.0,65668.1,NaN,9295.65,25697.80,10053,12
2,10053_18,13235.70,507386.0,7126.96,24525.7,NaN,2372.71,126506.0,16289.6,168107.0,...,111107.0,NaN,37932.6,245188.0,59986.1,10813.3,NaN,29102.70,10053,18
3,10138_12,12600.20,494581.0,9165.06,27193.5,22506.10,6015.90,156313.0,54546.4,204013.0,...,155619.0,14647.90,36927.7,229232.0,106564.0,26077.7,21441.80,7642.42,10138,12
4,10138_24,12003.20,522138.0,4498.51,17189.8,29112.40,2665.15,151169.0,52338.1,240892.0,...,177619.0,17061.10,25510.4,176722.0,59471.4,12639.2,15091.40,6168.55,10138,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,8699_24,9983.00,400290.0,24240.10,NaN,16943.50,6303.17,77493.6,46435.3,254247.0,...,NaN,6859.82,19106.7,121161.0,113872.0,14413.9,28225.50,8062.07,8699,24
1109,942_12,6757.32,360858.0,18367.60,14760.7,18603.40,1722.77,86847.4,37741.3,212132.0,...,94049.7,13415.70,21324.7,234094.0,82410.4,19183.7,17804.10,12277.00,942,12
1110,942_24,NaN,352722.0,22834.90,23393.1,16693.50,1487.91,114772.0,36095.7,185836.0,...,95949.9,11344.40,23637.6,256654.0,76931.9,19168.2,19215.90,14625.60,942,24
1111,942_48,11627.80,251820.0,22046.50,26360.5,22440.20,2117.43,82241.9,30146.6,167633.0,...,121696.0,19169.80,16724.9,232301.0,96905.9,21120.9,14089.80,16418.50,942,48


#### Clinical Data

In [42]:
clinical

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,6,NaN,NaN
1,55_3,55,3,10.0,7.0,19,NaN,NaN
2,55_6,55,6,8.0,10.0,58,NaN,NaN
3,55_9,55,9,8.0,9.0,17,0.0,On
4,55_12,55,12,10.0,10.0,2,0.0,On
...,...,...,...,...,...,...,...,...
2610,65043_48,65043,48,7.0,6.0,51,0.0,Off
2611,65043_54,65043,54,4.0,8.0,55,1.0,Off
2612,65043_60,65043,60,6.0,6.0,0,1.0,Off
2613,65043_72,65043,72,3.0,9.0,58,1.0,Off


In [45]:
# gather updrs columns into rows
df_clinical = clinical.melt(id_vars=['visit_id', 'patient_id', 'visit_month', 'upd23b_clinical_state_on_medication'], 
                   var_name='updrs', value_name='rating')

In [46]:
# Save new dfs to pickle
df_clinical.to_pickle("./df_clinical.pkl")  

# df_clinical = pd.read_pickle("./df_clinical.pkl")  
